# Introduction

Lets load our environment first

In [1]:
from func_adl_servicex import ServiceXSourceUpROOT
from hist import Hist
import awkward as ak

## Flat ROOT Files

ATLAS has distributed it's open data as flat ROOT files.

* On CERNOpenData they are a single zip file
* But they have been distributed as files available via EOS from CERN Open Data's EOS instance.

In [2]:
ggH125_ZZ4lep = 'root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root'
# ggH125_ZZ4lep = 'https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root'
ggH125_ZZ4lep_source = ServiceXSourceUpROOT([ggH125_ZZ4lep], 'mini','uproot-af')

* We use the `root://` address instead of `http://` due to efficiency and caching.
* `mini` is the tree name in the file
* last parameter basically describes the type of file - this is a flat root file that can be opened by the `uproot` python package.

Now that we have a reference to the datasource, lets pick out a single column and bring its contents back to our local instance:

In [3]:
from servicex import ignore_cache

with ignore_cache():
    r = (ggH125_ZZ4lep_source
         .Select(lambda e: {'lep_pt': e['lep_pt']})
         .AsAwkwardArray()
         .value()
        )
r

ClientConnectorCertificateError: Cannot connect to host uproot-atlas.servicex.af.uchicago.edu:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)')]

This is a standard awkward array - and it's shape is simply a series of lepton $p_T$'s for each of the 164716 events:

In [ ]:
r.type

Lets plot this. We'll use the `Hist` library, which is a nice wrapper around the `boost_histogram` library:
* Note we have a single axis, which is the muon's $p_T$

In [ ]:
h = (Hist.new
     .Reg(50, 0, 200, name='lep_pt', label='Lepton $p_T$ [GeV]')
     .Int64()
     )
h.fill(ak.flatten(r['lep_pt'])/1000.0)
_ = h.plot()

Lets get back a $p_T$ and an $\eta$ for the leptons now. This requires two things coming back:

In [ ]:
r = (ggH125_ZZ4lep_source
     .Select(lambda e: {'lep_pt': e['lep_pt'], 'lep_eta': e['lep_eta']})
     .AsAwkwardArray()
     .value()
    )
r.type

Note that it is columnar data:

* Each event contains two arrays
* The arrays are lepton pt and eta - not a tuple of lepton ($p_T$, $\eta$).

What if we want to cut on the eta? How do we relate these two columns? We use the zip function.

In [ ]:
r_cut = (ggH125_ZZ4lep_source
         .Select(lambda e: Zip({'pt': e['lep_pt'], 'eta': e['lep_eta']}))
         .Select(lambda leps: leps.Where(lambda l: abs(l.eta) < 1.0))
         .AsAwkwardArray()
         .value()
        )
r_cut.type

Interesting - same number of events - but we cut? Lets look at a histogram:

In [ ]:
h = (Hist.new
     .Reg(50, 0, 200, name='lep_pt', label='Lepton Track $p_T$ [GeV]')
     .StrCat([], name='cut', label='Cut Type', growth=True)
     .Int64()
     )
h.fill(lep_pt=ak.flatten(r['lep_pt'])/1000.0, cut='All')
h.fill(lep_pt=ak.flatten(r_cut['pt'])/1000.0, cut='eta')
artists = h.plot()

ax = artists[0].stairs.axes  # get the axis
ax.legend(loc="best");

In [ ]:
h = (Hist.new
     .Reg(8, 0, 8, name='muon_count', label='Number of leptons')
     .StrCat([], name='cut', label='Cut Type', growth=True)
     .Int64()
     )
h.fill(lepton_count=ak.count(r['lep_pt'], axis=-1), cut='All')
h.fill(lepton_count=ak.count(r_cut['pt'], axis=-1), cut='eta')
artists = h.plot()

ax = artists[0].stairs.axes  # get the axis
ax.legend(loc="best");

* We cut the number of leptons per event
* We now have some events with zero leptons - but those events were still returned
* We could remove them by doing a cut on the number of leptons...